In [18]:
import pandas as pd
import numpy as np
import ast

In [19]:
df = pd.read_csv("movies_transformado.csv")
# Muestra todas las columnas
pd.set_option('display.max_columns', None)

In [20]:
# Función para transformar las cadenas en el typo de dato correspondiente
def transformar_cadena(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return cadena

In [21]:
lista_columnas = df.columns.tolist()
for columna in lista_columnas:
    df[columna] = df[columna].apply(transformar_cadena)

In [22]:
# Ordenamos por vote_average y tomamos las primeras 5000 filas de cada uno
df_vote_average = df.sort_values(by='vote_average', ascending=False).head(5500).reset_index(drop=True)

In [23]:
print("df_vote_average: ",df_vote_average.shape)

df_vote_average:  (5500, 21)


In [24]:
# Aseguramos que los datos de la columna 'overview', 'genres' y 'production_companies' sean strings
df_vote_average['overview'] = df_vote_average['overview'].fillna('').astype('str')
df_vote_average['genres'] = df_vote_average['genres'].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else '')
df_vote_average['production_companies'] = df_vote_average['production_companies'].apply(lambda x: ' '.join(map(str, x)) if isinstance(x, list) else x)

In [25]:
# Reemplazar los valores nulos con cadenas vacias
df_vote_average['production_companies'] = df_vote_average['production_companies'].fillna('')

In [26]:
# Creamos una nueva columna llamada 'combined_features' que es una combinacion de las columnas 'overview', 'genres' y 
# 'production_companies'. Esta columna se usara para calcular la similitud entre diferentes peliculas.
df_vote_average['combined_features'] = df_vote_average['overview'] + ' ' + df_vote_average['genres'] + ' ' + df_vote_average['production_companies']

# Convertimos todos los textos a minusculas para evitar duplicados
df_vote_average['combined_features'] = df_vote_average['combined_features'].str.lower()



In [27]:
# Inicializamos el HashingVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
hash_vectorizer = HashingVectorizer(stop_words='english', n_features=4000)

In [28]:
# Aprende el vocabulario de 'combined_features' y transforma 'combined_features' en una matriz de vectores
hash_matrix = hash_vectorizer.fit_transform(df_vote_average['combined_features'])

In [29]:
df_vote_average.shape

(5500, 22)

In [30]:
df_vote_average.to_csv('movies_final.csv', index=False)

In [31]:
#para convertir las caracteristicas combinadas en una matriz
from sklearn.feature_extraction.text import CountVectorizer 
# para crear un modelo que pueda encontrar los vecinos mas cercanos en un espacio dado
from sklearn.neighbors import NearestNeighbors 

# Nos aseguramos de que estamos trabajando con los datos exportados del df_vote_average
df_movie_final= pd.read_csv('movies_final.csv')

In [32]:
#  En esta matriz, cada fila representa una película y cada columna 
# representa un termino en las caracteristicas combinadas
cv = CountVectorizer(stop_words='english', max_features=6000)
count_matrix = cv.fit_transform(df_movie_final['combined_features'])

In [33]:
# Creamos un modelo para encontrar los vecinos mas cercanos en un espacio de caracterisicaa
nn = NearestNeighbors(metric='cosine', algorithm='auto')
nn.fit(count_matrix)

# Creamos un indice de titulos de peliculas y eliminamos los duplicados
indices = pd.Series(df_vote_average.index, index=df_movie_final['title']).drop_duplicates()

In [34]:
def recomendacion(title):
    # Verifica si el titulo ingresado se encuentra en el df
    if title not in df_movie_final['title'].values:
        return 'La pelicula no se encuentra en el data set'
    else:
        index = indices[title]
        # Obtiene las puntuaciones de similitud de las 5 peliculas más cercanas
        distances, indices_knn = nn.kneighbors(count_matrix[index], n_neighbors=6)  # indica que queremos encontrar las 6 peliculas más similares, incluyendo la pelicula dada
        # Obtiene los indices de las peliculas
        movie_indices = indices_knn[0][1:]  # Se omite el primer indice (la pelicula misma) con [1:]
        # Devuelve las 5 peliculas mas similares
        return df_movie_final['title'].iloc[movie_indices].tolist()

print(recomendacion('Toy Story'))
print(recomendacion('Scarface'))
print(recomendacion('Redemption'))
print(recomendacion('Jumanji'))

['Toy Story 3', 'Приключения Домовёнка', 'Lou', 'Partysaurus Rex', "Geri's Game"]
['OSS 117: Mission for a Killer', 'Cocaine Cowboys: Reloaded', 'Cocaine Cowboys', 'Extraordinary Mission', 'Scarface']
['Dead Times', 'Daddy', 'Wolf Children', 'Ahí está el detalle', 'Dumbland']
La pelicula no se encuentra en el data set
